$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
$$

In [87]:
from os.path import dirname, join as pjoin
import scipy.io as sio

import unittest
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import scipy.fftpack as scifft

from fastMRI.data import transforms as Ttorch

%load_ext autoreload
%autoreload 2

test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cpu


In [9]:
data_dir = pjoin(dirname(sio.__file__), 'data')
print(data_dir)
mat_fname = pjoin(data_dir, 'check_fft.mat')
print(mat_fname)
mat_contents = sio.loadmat(mat_fname)

C:\Users\apahima\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\io\data
C:\Users\apahima\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\io\data\check_fft.mat


In [10]:
print(mat_contents.keys())

dict_keys(['__header__', '__version__', '__globals__', 'checkfft'])


In [166]:
check_fft_tensor = Ttorch.to_tensor(mat_contents['checkfft'])

print(check_fft_tensor[:,:,0])

tensor([[-1.1590e-03,  0.0000e+00,  9.8028e-04,  6.2628e-04,  4.7608e-04],
        [-9.8460e-04,  1.1812e-04,  4.5878e-04, -2.0897e-04,  7.7709e-04],
        [-4.3921e-04, -4.1387e-04, -9.0951e-04, -9.9373e-05,  9.9443e-04],
        [ 1.0274e-03,  1.7006e-03,  1.6781e-03,  1.7775e-03,  1.2988e-03],
        [ 1.3584e-03,  1.4071e-03,  1.8340e-03,  1.4109e-03,  1.2076e-03]],
       dtype=torch.float64)


In [169]:
a = Ttorch.ifft2(check_fft_tensor)

print(a[:,:,1])

tensor([[ 1.2866e-05, -3.3567e-05,  8.0284e-06, -5.3144e-07,  5.0853e-05],
        [ 2.9108e-05,  4.6276e-05, -1.1123e-04,  1.3407e-04, -2.2491e-05],
        [ 9.6077e-05, -2.2897e-05,  1.1187e-05, -3.9913e-04,  2.2122e-05],
        [-2.2319e-04,  1.2086e-04, -3.2155e-04, -4.5049e-04, -3.5900e-05],
        [-1.6459e-04,  1.1105e-04, -2.8516e-05,  4.5394e-04,  5.2527e-08]],
       dtype=torch.float64)


In [168]:
b_ifftshift_ifft2_ifftshift = np.fft.ifftshift(np.fft.ifft2(np.fft.ifftshift(mat_contents['checkfft'])))
b_ifftshift_ifft2_ifftshift_sci = scifft.ifftshift(scifft.ifft2(scifft.ifftshift(mat_contents['checkfft'])))


print('b_ifftshift_ifft2_ifftshift')
print(b_ifftshift_ifft2_ifftshift[:,:])
print('b_ifftshift_ifft2_ifftshift with SciPy')
print(b_ifftshift_ifft2_ifftshift_sci[:,:])

b_ifftshift_ifft2_ifftshift
[[-2.47216813e-05+1.28663389e-05j -2.90198280e-05-3.35667087e-05j
  -2.95610045e-06+8.02840640e-06j -4.77130354e-05-5.31437743e-07j
  -1.45292612e-04+5.08526227e-05j]
 [-8.62641410e-06+2.91082802e-05j  3.42320818e-05+4.62760036e-05j
  -1.14757219e-04-1.11229137e-04j -3.15073166e-04+1.34068506e-04j
  -5.35261405e-05-2.24905477e-05j]
 [ 1.19853759e-04+9.60767627e-05j -2.05750985e-05-2.28971844e-05j
   5.96008935e-05+1.11866209e-05j -1.80548528e-04-3.99130167e-04j
  -2.18450191e-04+2.21221387e-05j]
 [-1.17363295e-04-2.23185170e-04j  1.44423773e-04+1.20859465e-04j
  -1.59554625e-04-3.21545447e-04j  5.96677979e-04-4.50493318e-04j
   3.44145453e-04-3.59003732e-05j]
 [-2.31995933e-04-1.64592724e-04j  4.70683635e-06+1.11050511e-04j
  -1.33649912e-04-2.85158050e-05j -2.26849530e-04+4.53938450e-04j
  -1.82474395e-04+5.25274802e-08j]]
b_ifftshift_ifft2_ifftshift with SciPy
[[-2.47216813e-05+1.28663389e-05j -2.90198280e-05-3.35667087e-05j
  -2.95610045e-06+8.02840640e-0

In [89]:
b_ifftshift = (np.fft.ifftshift(mat_contents['checkfft']))
b_ifftshift_sci = (scifft.ifftshift(mat_contents['checkfft']))

print('b_ifftshift')
print(b_ifftshift[:,:])
print('b_ifftshift with SciPy fft')
print(b_ifftshift_sci[:,:])


b_ifftshift
[[-9.09506928e-04-7.17591386e-04j -9.93725425e-05-2.29483908e-04j
   9.94431122e-04-3.58467961e-04j -4.39214241e-04+5.08475161e-04j
  -4.13868809e-04-5.13671740e-04j]
 [ 1.67810949e-03-5.95581390e-04j  1.77746274e-03-9.62760514e-04j
   1.29884857e-03-1.29846889e-03j  1.02742749e-03+3.78005518e-04j
   1.70058103e-03-7.94724010e-04j]
 [ 1.83397742e-03-4.29899933e-04j  1.41093285e-03-7.39829392e-04j
   1.20758205e-03-8.79423542e-04j  1.35838303e-03+9.86356862e-04j
   1.40710534e-03+2.02399577e-04j]
 [ 9.80283044e-04-1.44885137e-03j  6.26279679e-04-1.42424421e-03j
   4.76082270e-04-7.14220728e-04j -1.15899738e-03-8.79765285e-05j
   0.00000000e+00+0.00000000e+00j]
 [ 4.58783409e-04-1.35940014e-03j -2.08965012e-04-6.60379872e-04j
   7.77090307e-04-4.39297534e-04j -9.84603130e-04-2.43219263e-05j
   1.18117690e-04+3.41024908e-04j]]
b_ifftshift with SciPy fft
[[-9.09506928e-04-7.17591386e-04j -9.93725425e-05-2.29483908e-04j
   9.94431122e-04-3.58467961e-04j -4.39214241e-04+5.0847516

In [90]:
b_ifft2_ifftshift = (np.fft.ifft2(np.fft.ifftshift(mat_contents['checkfft'])))
b_ifft2_ifftshift_sci = (scifft.ifft2(scifft.ifftshift(mat_contents['checkfft'])))

print('b_ifft2_ifftshift')
print(b_ifft2_ifftshift[:,:])
print('b_ifft2_ifftshift with Scipy')
print(b_ifft2_ifftshift_sci[:,:])

b_ifft2_ifftshift
[[ 5.96677979e-04-4.50493318e-04j  3.44145453e-04-3.59003732e-05j
  -1.17363295e-04-2.23185170e-04j  1.44423773e-04+1.20859465e-04j
  -1.59554625e-04-3.21545447e-04j]
 [-2.26849530e-04+4.53938450e-04j -1.82474395e-04+5.25274802e-08j
  -2.31995933e-04-1.64592724e-04j  4.70683635e-06+1.11050511e-04j
  -1.33649912e-04-2.85158050e-05j]
 [-4.77130354e-05-5.31437743e-07j -1.45292612e-04+5.08526227e-05j
  -2.47216813e-05+1.28663389e-05j -2.90198280e-05-3.35667087e-05j
  -2.95610045e-06+8.02840640e-06j]
 [-3.15073166e-04+1.34068506e-04j -5.35261405e-05-2.24905477e-05j
  -8.62641410e-06+2.91082802e-05j  3.42320818e-05+4.62760036e-05j
  -1.14757219e-04-1.11229137e-04j]
 [-1.80548528e-04-3.99130167e-04j -2.18450191e-04+2.21221387e-05j
   1.19853759e-04+9.60767627e-05j -2.05750985e-05-2.28971844e-05j
   5.96008935e-05+1.11866209e-05j]]
b_ifft2_ifftshift with Scipy
[[ 5.96677979e-04-4.50493318e-04j  3.44145453e-04-3.59003732e-05j
  -1.17363295e-04-2.23185170e-04j  1.44423773e-04+1

### Torch fft2

In [55]:
a_ifftshift = (Ttorch.ifftshift(check_fft_tensor))

print(a_ifftshift[:,:,1].numpy()+1j*a_ifftshift[:,:,0].numpy())

[[-9.09506928e-04-7.17591386e-04j -9.93725425e-05-2.29483908e-04j
   9.94431122e-04-3.58467961e-04j -4.39214241e-04+5.08475161e-04j
  -4.13868809e-04-5.13671740e-04j]
 [ 1.67810949e-03-5.95581390e-04j  1.77746274e-03-9.62760514e-04j
   1.29884857e-03-1.29846889e-03j  1.02742749e-03+3.78005518e-04j
   1.70058103e-03-7.94724010e-04j]
 [ 1.83397742e-03-4.29899933e-04j  1.41093285e-03-7.39829392e-04j
   1.20758205e-03-8.79423542e-04j  1.35838303e-03+9.86356862e-04j
   1.40710534e-03+2.02399577e-04j]
 [ 9.80283044e-04-1.44885137e-03j  6.26279679e-04-1.42424421e-03j
   4.76082270e-04-7.14220728e-04j -1.15899738e-03-8.79765285e-05j
   0.00000000e+00+0.00000000e+00j]
 [ 4.58783409e-04-1.35940014e-03j -2.08965012e-04-6.60379872e-04j
   7.77090307e-04-4.39297534e-04j -9.84603130e-04-2.43219263e-05j
   1.18117690e-04+3.41024908e-04j]]


In [60]:
a_ifft2_ifftshift = Ttorch.ifft2(Ttorch.ifftshift(check_fft_tensor))

print(a_ifft2_ifftshift[:,:,1].numpy()+1j*a_ifft2_ifftshift[:,:,0].numpy())

torch.Size([5, 5, 2])
torch.Size([5, 5, 2])
[[ 6.21880804e-05-1.24700633e-04j  1.73291600e-05+7.69479049e-04j
  -7.11935574e-05-2.27710083e-04j  3.55525719e-05-2.37878248e-05j
  -1.45099140e-04-1.67833543e-04j]
 [-1.14113834e-03+8.48896476e-04j -2.82188229e-04-8.67636155e-04j
   2.25171726e-03-1.16952556e-03j -6.68249560e-04-1.42579025e-04j
  -3.45408816e-04-4.35375707e-04j]
 [ 8.79972289e-04-9.02935685e-04j -1.49760615e-03-8.66198050e-04j
   2.98338990e-03-2.25246659e-03j  1.59041037e-03+8.31759379e-04j
   7.97867910e-04-5.00037707e-04j]
 [-2.02456120e-04-7.40880030e-04j -1.09225096e-03+1.10610694e-04j
   1.90334827e-03+1.08389663e-03j  1.45283989e-04-2.66134811e-04j
   7.70923064e-05-1.33218503e-04j]
 [-4.31320705e-05+1.45541401e-04j  2.82615848e-04-6.63332035e-05j
   1.50718819e-04+1.70540917e-03j  3.51616325e-04-7.17561474e-04j
  -2.46991727e-06-2.87795931e-04j]]


In [160]:
x = torch.tensor([[[4.1090, 2.6421],
         [1.6499, 2.2569],
         [3.6257, 2.6071]],

        [[2.7327, 2.0027],
         [1.5544, 1.9030],
         [1.9853, 1.8714]],

        [[2.9477, 3.2878],
         [1.9358, 2.7527],
         [2.0820, 0.8508]]])
print(x)
print(x.shape)
# x = x.permute(2,0,1)
print(x.shape)
print(x[:,:,0])

tensor([[[4.1090, 2.6421],
         [1.6499, 2.2569],
         [3.6257, 2.6071]],

        [[2.7327, 2.0027],
         [1.5544, 1.9030],
         [1.9853, 1.8714]],

        [[2.9477, 3.2878],
         [1.9358, 2.7527],
         [2.0820, 0.8508]]])
torch.Size([3, 3, 2])
torch.Size([3, 3, 2])
tensor([[4.1090, 1.6499, 3.6257],
        [2.7327, 1.5544, 1.9853],
        [2.9477, 1.9358, 2.0820]])


In [165]:
test =  torch.ifft(x,2)
print(test)
print('Convert to common view')
print(test[:,:,1].numpy() + 1j*test[:,:,0].numpy())

tensor([[[ 2.5136,  2.2416],
         [ 0.2224, -0.0444],
         [ 0.5271,  0.4469]],

        [[ 0.4145,  0.0635],
         [ 0.3401, -0.0698],
         [ 0.0393, -0.0568]],

        [[ 0.2001,  0.1969],
         [ 0.0289, -0.3862],
         [-0.1771,  0.2503]]])
Convert to common view
[[ 2.2416112 +2.513611j   -0.04435847+0.22240804j  0.4469474 +0.5271143j ]
 [ 0.0635176 +0.4145084j  -0.06981372+0.34014925j -0.05681898+0.03925212j]
 [ 0.19690475+0.20008045j -0.3861588 +0.02893681j  0.25026923-0.17706035j]]


In [163]:
test_np = np.fft.ifft2(x[:,:,1].numpy() + 1j*x[:,:,0].numpy())
print(test_np)

[[ 2.2416112 +2.513611j    0.4469474 +0.5271143j  -0.04435847+0.22240804j]
 [ 0.19690475+0.20008045j  0.25026923-0.17706035j -0.3861588 +0.02893681j]
 [ 0.0635176 +0.4145084j  -0.05681898+0.03925212j -0.06981372+0.34014925j]]
